In [1]:
import pandas as pd
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, precision_recall_curve, average_precision_score, mean_squared_error, r2_score, mean_absolute_error
from scipy.stats import pearsonr

import numpy as np

# preds

# Load the training set of meta-model
lipo_chemberta2_valid2 = pd.read_csv('./chemberta2/results/lipo/chemberta2_valid2_lipo_2_predictions.csv')
lipo_molformer_valid2 = pd.read_csv('./molformer/results/lipo/molformer_valid2_lipo_2.csv')
lipo_molbert_valid2 = pd.read_csv('./molbert/results/lipo/molbert_valid2_lipo_2.csv')

# Load the test data for each model
lipo_chemberta2_test = pd.read_csv('./chemberta2/results/lipo/chemberta2_test_lipo_2_predictions.csv')
lipo_molformer_test = pd.read_csv('./molformer/results/lipo/molformer_test_lipo_2.csv')
lipo_molbert_test = pd.read_csv('./molbert/results/lipo/molbert_test_lipo_2.csv')

train_mean = 2.1799722222222226
train_sd = 1.2045431891731355

# features

lipo_chemberta2_features_valid2 = pd.read_csv('./chemberta2/features/lipo/chemberta2_valid2_lipo_2_features.csv')
lipo_chemberta2_features_test = pd.read_csv('./chemberta2/features/lipo/chemberta2_test_lipo_2_features.csv')

lipo_molformer_features_valid2 = pd.read_csv('./molformer/features/lipo/molformer_valid2_lipo_2_features.csv')
lipo_molformer_features_test = pd.read_csv('./molformer/features/lipo/molformer_test_lipo_2_features.csv')

lipo_molbert_features_valid2 = pd.read_csv('./molbert/features/lipo/molbert_valid2_lipo_2_features.csv')
lipo_molbert_features_test = pd.read_csv('./molbert/features/lipo/molbert_test_lipo_2_features.csv')

For Lipo (Regression)

In [2]:
# Preparing the actual and predicted values
# Chemberta2
lipo_chemberta_actual = lipo_chemberta2_test['target'] 
lipo_chemberta_pred = lipo_chemberta2_test['pred_raw']

# Molformer
lipo_molformer_actual = lipo_molformer_test['target']
lipo_molformer_pred = lipo_molformer_test['pred_raw']

# molbert
lipo_molbert_actual = lipo_molbert_test['target_raw']
lipo_molbert_pred = lipo_molbert_test['pred_raw']

In [3]:
# Calculating metrics
lipo_metrics_results = {}

for model_name, actual, pred in [("Chemberta2", lipo_chemberta_actual, lipo_chemberta_pred),
                                 ("Molformer", lipo_molformer_actual, lipo_molformer_pred),
                                 ("Molbert", lipo_molbert_actual, lipo_molbert_pred)]:
    lipo_metrics_results[model_name] = {
        "MAE": mean_absolute_error(actual, pred),
        "RMSE": np.sqrt(mean_squared_error(actual, pred)),
        "R2 Score": r2_score(actual, pred),
        "Correlation": pearsonr(actual, pred)[0]  # Only record the correlation coefficient
    }

lipo_metrics_results

{'Chemberta2': {'MAE': 0.47080364152170984,
  'RMSE': 0.6194242164406615,
  'R2 Score': 0.725099924775132,
  'Correlation': 0.8597960575490352},
 'Molformer': {'MAE': 0.45518342900000003,
  'RMSE': 0.6064332745501879,
  'R2 Score': 0.7365097534650544,
  'Correlation': 0.867404233883593},
 'Molbert': {'MAE': 0.5100067769880953,
  'RMSE': 0.6507887204149233,
  'R2 Score': 0.6965560154119396,
  'Correlation': 0.8370118646634633}}

In [4]:
# standardized valid2 labels
lipo_y_ensemble_valid2 = (lipo_chemberta2_valid2['target'] - train_mean)/train_sd

# Create the features for the ensemble from the prediction probabilities of being in class 1
lipo_X_ensemble_valid2 = pd.concat([
    lipo_chemberta2_valid2['pred_z'] - lipo_y_ensemble_valid2,
    lipo_molformer_valid2['pred_z'] - lipo_y_ensemble_valid2, 
    lipo_molbert_valid2['pred_z'] - lipo_y_ensemble_valid2,
    # add features from training set
    lipo_chemberta2_valid2['pred_z'],
    lipo_molformer_valid2['pred_z'],
    lipo_molbert_valid2['pred_z']
], axis=1)

# change feature names of the ensemble so that they are unique
lipo_X_ensemble_valid2.columns = ['residuals_chemberta', 'residuals_molformer', 'residuals_molbert', 'chemberta', 'molformer', 'molbert']

In [5]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

# Set seed for reproducibility
seed = 0

# Compute residuals
chemberta_y_residual = lipo_chemberta2_valid2['pred_z'] - lipo_y_ensemble_valid2
molformer_y_residual = lipo_molformer_valid2['pred_z'] - lipo_y_ensemble_valid2
molbert_y_residual = lipo_molbert_valid2['pred_z'] - lipo_y_ensemble_valid2

# For input features, we use pred_z and the other features
chemberta_X = pd.concat([lipo_chemberta2_valid2['pred_z'], 
                         lipo_chemberta2_features_valid2.iloc[:, 2:]], axis=1)
molformer_X = pd.concat([lipo_molformer_valid2['pred_z'], 
                         lipo_molformer_features_valid2.iloc[:, 1:]], axis=1)
molbert_X = pd.concat([lipo_molbert_valid2['pred_z'], 
                       lipo_molbert_features_valid2.iloc[:, 1:]], axis=1)

# Standardize each dataset
scaler_chemberta = StandardScaler().fit(chemberta_X)
scaler_molformer = StandardScaler().fit(molformer_X)
scaler_molbert = StandardScaler().fit(molbert_X)

chemberta_X_scaled = scaler_chemberta.transform(chemberta_X)
molformer_X_scaled = scaler_molformer.transform(molformer_X)
molbert_X_scaled = scaler_molbert.transform(molbert_X)

# Initialize the Lasso models with a random_state for reproducibility
lasso_chemberta = Lasso(max_iter=10000, tol=0.0001, random_state=seed)
lasso_molformer = Lasso(max_iter=10000, tol=0.0001, random_state=seed)
lasso_molbert = Lasso(max_iter=10000, tol=0.0001, random_state=seed)

# Setup cross-validation for alpha tuning
param_grid = {
    'alpha': np.logspace(-4, 1, 10)  # Only alpha for Lasso
}

# Set up GridSearchCV for Lasso with neg_mean_squared_error scoring and random_state
cv_chemberta = GridSearchCV(estimator=lasso_chemberta, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
cv_molformer = GridSearchCV(estimator=lasso_molformer, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
cv_molbert = GridSearchCV(estimator=lasso_molbert, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)

# Fit GridSearchCV for each dataset with residual as the target
cv_chemberta.fit(chemberta_X_scaled, chemberta_y_residual)
cv_molformer.fit(molformer_X_scaled, molformer_y_residual)
cv_molbert.fit(molbert_X_scaled, molbert_y_residual)

# Retrieve the best models and parameters
best_model_chemberta = cv_chemberta.best_estimator_
best_model_molformer = cv_molformer.best_estimator_
best_model_molbert = cv_molbert.best_estimator_


/Users/lorrainelu/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.107e+01, tolerance: 1.891e-02
  model = cd_fast.enet_coordinate_descent(
/Users/lorrainelu/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.471e+01, tolerance: 1.949e-02
  model = cd_fast.enet_coordinate_descent(
/Users/lorrainelu/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regula

In [6]:
# ChemBERTa
chemberta_X_test = pd.concat([lipo_chemberta2_test['pred_z'], 
                              lipo_chemberta2_features_test.iloc[:, 2:]], axis=1)

# MolFormer
molformer_X_test = pd.concat([lipo_molformer_test['pred_z'], 
                              lipo_molformer_features_test.iloc[:, 1:]], axis=1)

# MolBERT
molbert_X_test = pd.concat([lipo_molbert_test['pred_z'], 
                            lipo_molbert_features_test.iloc[:, 1:]], axis=1)

# Scale the test data using the corresponding scalers
chemberta_X_test_scaled = scaler_chemberta.transform(chemberta_X_test)
molformer_X_test_scaled = scaler_molformer.transform(molformer_X_test)
molbert_X_test_scaled = scaler_molbert.transform(molbert_X_test)

# Predict residuals for the test set using the trained GroupLasso models
y_pred_residuals_chemberta_test = best_model_chemberta.predict(chemberta_X_test_scaled)
y_pred_residuals_molformer_test = best_model_molformer.predict(molformer_X_test_scaled)
y_pred_residuals_molbert_test = best_model_molbert.predict(molbert_X_test_scaled)

# Create a DataFrame to store the predicted residuals for the test set
predicted_residuals_test = pd.DataFrame({
    'Residuals_Chemberta': y_pred_residuals_chemberta_test,
    'Residuals_Molformer': y_pred_residuals_molformer_test,
    'Residuals_Molbert': y_pred_residuals_molbert_test
})

# Load test data into ensemble DataFrame, adding the predicted residuals and original predictions
lipo_X_ensemble_test = pd.concat([
    predicted_residuals_test,
    lipo_chemberta2_test['pred_z'],
    lipo_molformer_test['pred_z'],  
    lipo_molbert_test['pred_z']
], axis=1)

# Rename feature columns so that they are unique
lipo_X_ensemble_test.columns = ['residuals_chemberta', 'residuals_molformer', 'residuals_molbert', 'chemberta', 'molformer', 'molbert']

# True test labels
lipo_y_ensemble_test = lipo_chemberta2_test['target']

In [7]:
# scale the features
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
lipo_X_ensemble_valid2_scaled = scaler.fit_transform(lipo_X_ensemble_valid2)
lipo_X_ensemble_test_scaled = scaler.transform(lipo_X_ensemble_test)

lipo_X_ensemble_valid2_scaled = pd.DataFrame(lipo_X_ensemble_valid2_scaled, columns=lipo_X_ensemble_valid2.columns)
lipo_X_ensemble_test_scaled = pd.DataFrame(lipo_X_ensemble_test_scaled, columns=lipo_X_ensemble_test.columns)

In [8]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import pearsonr
import numpy as np

# Define the model with LASSO for regression
lasso_model = Lasso(random_state=0, max_iter=5000)

# Define the hyperparameter grid
# Use fewer discrete values for alpha
alphas = [0.01, 0.1, 1, 3]  # Reduced number of points focusing on lower and mid-range

params = {
    'alpha': alphas
}

# Perform GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(lasso_model, param_grid=params, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(lipo_X_ensemble_valid2_scaled, lipo_y_ensemble_valid2)

# Get the best hyperparameters
lipo_best_lasso_params = grid_search.best_params_
print(lipo_best_lasso_params)

# Initialize and train the best Lasso model
lipo_best_lasso_model = Lasso(alpha=lipo_best_lasso_params['alpha'], random_state=0, max_iter=5000)
lipo_best_lasso_model.fit(lipo_X_ensemble_valid2_scaled, lipo_y_ensemble_valid2)

# Predict the test set
lipo_lasso_pred = lipo_best_lasso_model.predict(lipo_X_ensemble_test_scaled) * train_sd + train_mean

# Calculate the metrics
lipo_lasso_metrics = {
    "MAE": mean_absolute_error(lipo_y_ensemble_test, lipo_lasso_pred),
    "RMSE": np.sqrt(mean_squared_error(lipo_y_ensemble_test, lipo_lasso_pred)),
    "R2 Score": r2_score(lipo_y_ensemble_test, lipo_lasso_pred),
    "Correlation": pearsonr(lipo_y_ensemble_test, lipo_lasso_pred)[0]
}

lipo_lasso_metrics


{'alpha': 0.01}


{'MAE': 0.5064893489898539,
 'RMSE': 0.6469818423193154,
 'R2 Score': 0.7000957067025788,
 'Correlation': 0.8382571437342055}

In [9]:
# create a table to record all metrics for lipo
lipo_metrics_results["Lasso"] = lipo_lasso_metrics

lipo_metrics_df = pd.DataFrame(lipo_metrics_results).T
# keep 3 digits after the decimal point
lipo_metrics_df = lipo_metrics_df.round(3)

# export table to csv
lipo_metrics_df.to_csv('./split2_lipo_metrics_lasso.csv')